In [4]:
import numpy as np
import pickle
import os

seed = 2023

In [19]:
from nlpsig_networks.scripts.ffn_baseline_functions import (
    ffn_hyperparameter_search,
)

In [22]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [2]:
%run ../load_anno_mi.py

In [3]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-06-27 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-27 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-27 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-06-27 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-06-27 00:00:34


In [5]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(13551, 384)

# Baseline: FFN baseline

Using the embeddings for the sentences directly in a FFN to predict the client talk type.

Going to try out some variations on the size of the hidden layer for a one-hidden layer FFN:

In [27]:
num_epochs = 100
hidden_dim_sizes = [32, 64, 128, 256]
dropout_rates = [0.5, 0.2, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [0, 1, 12, 123, 1234]
loss = "focal"
gamma = 2
validation_metric = "f1"

In [28]:
hidden_dim_sizes

[32, 64, 128, 256]

In [29]:
learning_rates

[0.001, 0.0001, 0.0005]

We use the `ffn_hyperparameter_search` function which loops through the different hidden dimensions, dropout rates and learning rates to find the best model for the validation set. We evaluate the model on several seeds and average the performance over the seeds.

In [32]:
ffn_current, best_ffn_current, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data_client,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=False,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_current_focal_{gamma}.csv",
    verbose=False
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_best_model.csv


In [33]:
ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.690706,0.607652,"[0.7906705539358602, 0.5107794361525705, 0.521...",0.540033,0.713755,0.616275,"[0.8123209169054441, 0.5584158415841585, 0.478...",32,0.5,0.0010,0,2,False,0
0,focal,0.697398,0.622115,"[0.7928994082840236, 0.5337620578778135, 0.539...",0.525517,0.708178,0.615140,"[0.8063583815028901, 0.5546875, 0.484375]",32,0.5,0.0010,1,2,False,0
0,focal,0.701115,0.628896,"[0.7931644077784326, 0.5331125827814569, 0.560...",0.519634,0.711896,0.613063,"[0.8134863701578193, 0.5524193548387096, 0.473...",32,0.5,0.0010,12,2,False,0
0,focal,0.695167,0.620094,"[0.790946992257296, 0.5405405405405406, 0.5287...",0.520470,0.716543,0.625157,"[0.813362381989833, 0.5681818181818183, 0.4939...",32,0.5,0.0010,123,2,False,0
0,focal,0.707063,0.634626,"[0.8023668639053255, 0.5300813008130082, 0.571...",0.520771,0.710037,0.617372,"[0.8103821196827686, 0.5436507936507937, 0.498...",32,0.5,0.0010,1234,2,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.685502,0.621982,"[0.7767695099818512, 0.5285285285285285, 0.560...",0.538714,0.707249,0.623856,"[0.7997043606799704, 0.5794392523364486, 0.492...",256,0.1,0.0005,0,2,False,35
0,focal,0.693680,0.622106,"[0.7903903903903904, 0.5318471337579618, 0.544...",0.524079,0.712825,0.625979,"[0.8093498904309716, 0.5685884691848907, 0.5]",256,0.1,0.0005,1,2,False,35
0,focal,0.703346,0.620653,"[0.8009395184967704, 0.544891640866873, 0.5161...",0.517208,0.718401,0.621770,"[0.8143369175627241, 0.5644859813084112, 0.486...",256,0.1,0.0005,12,2,False,35
0,focal,0.686245,0.615210,"[0.7795795795795796, 0.5385779122541604, 0.527...",0.542989,0.714684,0.632609,"[0.8046989720998531, 0.5855513307984791, 0.507...",256,0.1,0.0005,123,2,False,35


In [49]:
ffn_current.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_72642/3305696319.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_current.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  valid_loss  \
hidden_dim dropout_rate learning_rate                                   
32         0.1          0.0001         0.680149  0.594394    0.536618   
                        0.0005         0.689963  0.614788    0.526840   
                        0.0010         0.690706  0.618284    0.530352   
           0.2          0.0001         0.684312  0.590173    0.559358   
                        0.0005         0.689071  0.607333    0.527839   
                        0.0010         0.698439  0.625117    0.528452   
           0.5          0.0001         0.687732  0.602205    0.534753   
                        0.0005         0.696952  0.616860    0.523067   
                        0.0010         0.698290  0.622677    0.525285   
64         0.1          0.0001         0.685651  0.603257    0.531592   
                        0.0005         0.696803  0.620145    0.526482   
                        0.0010         0.692045  0.619158    0.540684   
           0.2          0.0001         0.682082  0.597065    0.534656   
                        0.0005         0.694126  0.616464    0.524993   
                        0.0010         0.691599  0.620355    0.533276   
           0.5          0.0001         0.691152  0.610931    0.527755   
                        0.0005         0.697695  0.619066    0.523802   
                        0.0010         0.700372  0.626777    0.521953   
128        0.1          0.0001         0.685799  0.607107    0.526089   
                        0.0005         0.695167  0.624748    0.526609   
                        0.0010         0.693383  0.621810    0.533448   
           0.2          0.0001         0.684015  0.602212    0.524489   
                        0.0005         0.691450  0.623150    0.530421   
                        0.0010         0.692639  0.618072    0.546021   
           0.5          0.0001         0.686245  0.604447    0.525812   
                        0.0005         0.689814  0.618327    0.524048   
                        0.0010         0.697249  0.623825    0.536209   
256        0.1          0.0001         0.689665  0.612789    0.522788   
                        0.0005         0.691896  0.618501    0.530330   
                        0.0010         0.695613  0.619298    0.534706   
           0.2          0.0001         0.687881  0.607708    0.522996   
                        0.0005         0.693086  0.621486    0.529968   
                        0.0010         0.695762  0.622658    0.516973   
           0.5          0.0001         0.686840  0.604744    0.529126   
                        0.0005         0.697249  0.624573    0.522070   
                        0.0010         0.694870  0.619210    0.531376   

                                       valid_accuracy  valid_f1   seed  gamma  \
hidden_dim dropout_rate learning_rate                                           
32         0.1          0.0001               0.697955  0.596497  274.0    2.0   
                        0.0005               0.701859  0.608003  274.0    2.0   
                        0.0010               0.704461  0.617942  274.0    2.0   
           0.2          0.0001               0.698513  0.586144  274.0    2.0   
                        0.0005               0.706691  0.608933  274.0    2.0   
                        0.0010               0.710595  0.618819  274.0    2.0   
           0.5          0.0001               0.707435  0.606277  274.0    2.0   
                        0.0005               0.714312  0.614470  274.0    2.0   
                        0.0010               0.712082  0.617401  274.0    2.0   
64         0.1          0.0001               0.700372  0.599154  274.0    2.0   
                        0.0005               0.706691  0.610342  274.0    2.0   
                        0.0010               0.708550  0.621389  274.0    2.0   
           0.2          0.0001               0.702602  0.600271  274.0    2.0   
                        0.0005               0.70

In [34]:
best_ffn_current

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.695911,0.623336,"[0.7902843601895735, 0.5279999999999999, 0.551...",0.523019,0.717472,0.630944,"[0.810655147588193, 0.5633802816901409, 0.5187...",256,0.5,0.0005,0,2,False
0,focal,0.698885,0.629996,"[0.7909738717339667, 0.5314465408805031, 0.567...",0.522244,0.723048,0.635308,"[0.8167994207096306, 0.5752895752895753, 0.513...",256,0.5,0.0005,1,2,False
0,focal,0.708550,0.632592,"[0.8021139166177335, 0.5448613376835236, 0.550...",0.514363,0.721190,0.627238,"[0.816034359341446, 0.5697445972495089, 0.4959...",256,0.5,0.0005,12,2,False
0,focal,0.689963,0.613781,"[0.7871581450653984, 0.529595015576324, 0.5245...",0.533217,0.717472,0.630246,"[0.8101449275362318, 0.5747572815533981, 0.505...",256,0.5,0.0005,123,2,False
0,focal,0.692937,0.623159,"[0.7877697841726619, 0.5312977099236642, 0.550...",0.517509,0.714684,0.628434,"[0.8084179970972424, 0.564885496183206, 0.5120...",256,0.5,0.0005,1234,2,False


In [35]:
best_ffn_current["f1"].mean()

0.624572893977324

In [36]:
np.stack(best_ffn_current["f1_scores"]).mean(axis=0)

array([0.79166002, 0.53304012, 0.54901855])

## KFold

We can repeat this but use K-Fold evaluation instead - by default, we have $K=5$ folds.

In [37]:
ffn_current_kfold, best_ffn_current_kfold, _, __ = ffn_hyperparameter_search(
    num_epochs=num_epochs,
    x_data=sbert_embeddings[client_index],
    y_data=y_data_client,
    hidden_dim_sizes=hidden_dim_sizes,
    output_dim=output_dim_client,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    k_fold=True,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/ffn_current_focal_{gamma}_kfold.csv",
    verbose=False
)

  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/ffn_current_focal_2_kfold_best_model.csv


In [38]:
ffn_current_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold,model_id
0,focal,0.684015,0.592274,"[0.7878151260504201, 0.5247198417930125, 0.464...",None,0.684015,0.592274,"[0.7878151260504201, 0.5247198417930125, 0.464...",32,0.5,0.0010,0,2,True,0
0,focal,0.687286,0.595535,"[0.79050736497545, 0.5316537467700257, 0.46444...",None,0.687286,0.595535,"[0.79050736497545, 0.5316537467700257, 0.46444...",32,0.5,0.0010,1,2,True,0
0,focal,0.687435,0.594120,"[0.7912011173184357, 0.5291982843945893, 0.461...",None,0.687435,0.594120,"[0.7912011173184357, 0.5291982843945893, 0.461...",32,0.5,0.0010,12,2,True,0
0,focal,0.681041,0.589237,"[0.786334820380371, 0.5237020316027089, 0.4576...",None,0.681041,0.589237,"[0.786334820380371, 0.5237020316027089, 0.4576...",32,0.5,0.0010,123,2,True,0
0,focal,0.677323,0.588665,"[0.7818396226415094, 0.5186841264771639, 0.465...",None,0.677323,0.588665,"[0.7818396226415094, 0.5186841264771639, 0.465...",32,0.5,0.0010,1234,2,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,focal,0.680297,0.595598,"[0.7828270748624732, 0.5393958268452196, 0.464...",None,0.680297,0.595598,"[0.7828270748624732, 0.5393958268452196, 0.464...",256,0.1,0.0005,0,2,True,35
0,focal,0.685651,0.594656,"[0.7887521968365554, 0.531109685695959, 0.4641...",None,0.685651,0.594656,"[0.7887521968365554, 0.531109685695959, 0.4641...",256,0.1,0.0005,1,2,True,35
0,focal,0.684312,0.597738,"[0.7854356306892067, 0.538022813688213, 0.4697...",None,0.684312,0.597738,"[0.7854356306892067, 0.538022813688213, 0.4697...",256,0.1,0.0005,12,2,True,35
0,focal,0.689665,0.600595,"[0.7915784521289109, 0.5396825396825398, 0.470...",None,0.689665,0.600595,"[0.7915784521289109, 0.5396825396825398, 0.470...",256,0.1,0.0005,123,2,True,35


In [50]:
ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()

/var/folders/l7/bnn1j6bs3bqfskq2jlq55yh80000gr/T/ipykernel_72642/3418571787.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ffn_current_kfold.groupby(["hidden_dim", "dropout_rate", "learning_rate"]).mean()


accuracy        f1  valid_accuracy  \
hidden_dim dropout_rate learning_rate                                       
32         0.1          0.0001         0.679227  0.588580        0.679227   
                        0.0005         0.679435  0.590432        0.679435   
                        0.0010         0.677145  0.589795        0.677145   
           0.2          0.0001         0.678662  0.587456        0.678662   
                        0.0005         0.679197  0.590157        0.679197   
                        0.0010         0.677799  0.590549        0.677799   
           0.5          0.0001         0.682230  0.589516        0.682230   
                        0.0005         0.682349  0.591153        0.682349   
                        0.0010         0.683420  0.591966        0.683420   
64         0.1          0.0001         0.679316  0.588534        0.679316   
                        0.0005         0.678364  0.590378        0.678364   
                        0.0010         0.677829  0.591837        0.677829   
           0.2          0.0001         0.680387  0.590265        0.680387   
                        0.0005         0.680862  0.592269        0.680862   
                        0.0010         0.678989  0.592403        0.678989   
           0.5          0.0001         0.681844  0.590885        0.681844   
                        0.0005         0.681695  0.592455        0.681695   
                        0.0010         0.681279  0.592656        0.681279   
128        0.1          0.0001         0.680892  0.592045        0.680892   
                        0.0005         0.679554  0.591535        0.679554   
                        0.0010         0.680625  0.594072        0.680625   
           0.2          0.0001         0.680268  0.591865        0.680268   
                        0.0005         0.681071  0.593754        0.681071   
                        0.0010         0.680327  0.593437        0.680327   
           0.5          0.0001         0.682439  0.593287        0.682439   
                        0.0005         0.681874  0.593557        0.681874   
                        0.0010         0.683063  0.595426        0.683063   
256        0.1          0.0001         0.683480  0.594105        0.683480   
                        0.0005         0.685115  0.597005        0.685115   
                        0.0010         0.682617  0.595745        0.682617   
           0.2          0.0001         0.684729  0.595299        0.684729   
                        0.0005         0.684848  0.596344        0.684848   
                        0.0010         0.682825  0.594917        0.682825   
           0.5          0.0001         0.685145  0.595145        0.685145   
                        0.0005         0.683004  0.596287        0.683004   
                        0.0010         0.683807  0.596170        0.683807   

                                       valid_f1   seed  gamma  k_fold  \
hidden_dim dropout_rate learning_rate                                   
32         0.1          0.0001         0.588580  274.0    2.0     1.0   
                        0.0005         0.590432  274.0    2.0     1.0   
                        0.0010         0.589795  274.0    2.0     1.0   
           0.2          0.0001         0.587456  274.0    2.0     1.0   
                        0.0005         0.590157  274.0    2.0     1.0   
                        0.0010         0.590549  274.0    2.0     1.0   
           0.5          0.0001         0.589516  274.0    2.0     1.0   
                        0.0005         0.591153  274.0    2.0     1.0   
                        0.0010         0.591966  274.0    2.0     1.0   
64         0.1          0.0001         0.588534  274.0    2.0     1.0   
                        0.0005         0.590378  274.0    2.0     1.0   
                        0.0010         0.591837  274.0    2.0     1.0   
           0.2          0.0001         0.590265  274.0    2.0     1.0   
                 

In [39]:
best_ffn_current_kfold

,loss,accuracy,f1,f1_scores,valid_loss,valid_accuracy,valid_f1,valid_f1_scores,hidden_dim,dropout_rate,learning_rate,seed,gamma,k_fold
0,focal,0.680297,0.595598,"[0.7828270748624732, 0.5393958268452196, 0.464...",None,0.680297,0.595598,"[0.7828270748624732, 0.5393958268452196, 0.464...",256,0.1,0.0005,0,2,True
0,focal,0.685651,0.594656,"[0.7887521968365554, 0.531109685695959, 0.4641...",None,0.685651,0.594656,"[0.7887521968365554, 0.531109685695959, 0.4641...",256,0.1,0.0005,1,2,True
0,focal,0.684312,0.597738,"[0.7854356306892067, 0.538022813688213, 0.4697...",None,0.684312,0.597738,"[0.7854356306892067, 0.538022813688213, 0.4697...",256,0.1,0.0005,12,2,True
0,focal,0.689665,0.600595,"[0.7915784521289109, 0.5396825396825398, 0.470...",None,0.689665,0.600595,"[0.7915784521289109, 0.5396825396825398, 0.470...",256,0.1,0.0005,123,2,True
0,focal,0.685651,0.596440,"[0.7886064030131827, 0.533974358974359, 0.4667...",None,0.685651,0.596440,"[0.7886064030131827, 0.533974358974359, 0.4667...",256,0.1,0.0005,1234,2,True


In [52]:
best_ffn_current_kfold["f1"].mean()

0.5970053260201187

In [41]:
np.stack(best_ffn_current_kfold["f1_scores"]).mean(axis=0)

array([0.78743995, 0.53643704, 0.46713898])